# APIM ❤️ AI Agents

## MCP-enabled A2A agents lab (initial release)
![flow](../../images/model-context-protocol.gif)

Playground to experiment the [Model Context Protocol](https://modelcontextprotocol.io/) with Azure API Management to enable plug & play of tools to LLMs. Leverages the [credential manager](https://learn.microsoft.com/en-us/azure/api-management/credentials-overview) for  managing OAuth 2.0 tokens to backend tools and [client token validation](https://learn.microsoft.com/en-us/azure/api-management/validate-jwt-policy) to ensure end-to-end authentication and authorization.   
This lab includes the following MCP servers:
- Basic oncall service: provides a tool to get a list of random people currently on-call with their status and time zone.
- Basic weather service: provide tools to get cities for a given country and retrieve random weather information for a specified city.
- GitHub Issues MCP Server: provide tools to authenticate on GitHub using the APIM Credential Manager, retrieves user information, and lists issues for a specified repository. This [sequence diagram](./diagrams/diagrams.md) explains the flow. 
- ServiceNow incidents MCP Server: provides tools to authenticates on ServiceNow using the APIM Credential Manager, lists incidents, retrieves a particular incident and create a new one.

### Prerequisites

- [Python 3.13 or later](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Python environment](https://code.visualstudio.com/docs/python/environments#_creating-environments) with the [requirements.txt](../../requirements.txt) or run `pip install -r requirements.txt` in your terminal
- [An Azure Subscription](https://azure.microsoft.com/free/) with [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#contributor) + [RBAC Administrator](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#role-based-access-control-administrator) or [Owner](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#owner) roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and [Signed into your Azure subscription](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

▶️ Click `Run All` to execute all steps sequentially, or execute them `Step by Step`...


<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [19]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"

apim_sku = 'Basicv2'

openai_resources = [ {"name": "openai1", "location": "uksouth"}]
openai_model_name = "gpt-4o-mini"
openai_model_version = "2024-07-18"
openai_model_sku = "GlobalStandard"
openai_deployment_name = "gpt-4o-mini"
openai_api_version = "2024-10-21"

build = 0
weather_mcp_server_image = "weather-mcp-server"
weather_mcp_server_src = "src/weather/mcp-server"

oncall_mcp_server_image = "oncall-mcp-server"
oncall_mcp_server_src = "src/oncall/mcp-server"

github_mcp_server_image = "github-mcp-server"
github_mcp_server_src = "src/github/mcp-server"

servicenow_mcp_server_image = "servicenow-mcp-server"
servicenow_mcp_server_src = "src/servicenow/mcp-server"
servicenow_instance_name = "" # Add here the name of your ServiceNow instance, e.g. "businessname-dev". Leave empty if you don't want to use ServiceNow.

utils.print_ok('Notebook initialized')

✅ Notebook initialized ⌚ 17:45:07.537009 


<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [20]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

⚙️ Running: az account show 
✅ Retrieved az account ⌚ 17:45:13.741746 :3s]
👉🏽 Current user: admin@MngEnv749975.onmicrosoft.com
👉🏽 Tenant ID: bb33d5b4-87eb-4a6e-bd80-5fb0bba89e65
👉🏽 Subscription ID: 77f5679a-f7cd-4b09-91f0-e8808bf4447e


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [21]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "openAIConfig": { "value": openai_resources },
        "openAIDeploymentName": { "value": openai_deployment_name },
        "openAIModelName": { "value": openai_model_name },
        "openAIModelVersion": { "value": openai_model_version },
        "openAIModelSKU": { "value": openai_model_sku },
        "openAIAPIVersion": { "value": openai_api_version },
        "serviceNowInstanceName": { "value": servicenow_instance_name }
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

⚙️ Running: az group show --name lab-mcp-a2a-agents 
👉🏽 Using existing resource group 'lab-mcp-a2a-agents'
⚙️ Running: az deployment group create --name mcp-a2a-agents --resource-group lab-mcp-a2a-agents --template-file main.bicep --parameters params.json 
✅ Deployment 'mcp-a2a-agents' succeeded ⌚ 17:47:26.774404 :55s]


<a id='3'></a>
### 3️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [ ]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM Gateway URL')
    apim_resource_name = utils.get_deployment_output(output, 'apimResourceName', 'APIM Resource Name')
    apim_subscription_key = utils.get_deployment_output(output, 'apimSubscriptionKey', 'APIM Subscription Key (masked)', True)
    app_insights_name = utils.get_deployment_output(output, 'applicationInsightsName', 'Application Insights Name')
    container_registry_name = utils.get_deployment_output(output, 'containerRegistryName', 'Container Registry Name')
    weather_containerapp_resource_name = utils.get_deployment_output(output, 'weatherMCPServerContainerAppResourceName', 'Weather Container App Resource Name')
    oncall_containerapp_resource_name = utils.get_deployment_output(output, 'oncallMCPServerContainerAppResourceName', 'Oncall Container App Resource Name')
    github_containerapp_resource_name = utils.get_deployment_output(output, 'gitHubMCPServerContainerAppResourceName', 'GitHub Container App Resource Name')
    if servicenow_instance_name:
        servicenow_containerapp_resource_name = utils.get_deployment_output(output, 'servicenowMCPServerContainerAppResourceName', 'servicenow Container App Resource Name')
    
    a2a_weather_containerapp_resource_name = utils.get_deployment_output(output, 'a2AWeatherServerContainerAppResourceName', 'A2A (Weather) Container App Resource Name')
    a2a_oncall_containerapp_resource_name = utils.get_deployment_output(output, 'a2AOncallServerContainerAppResourceName', 'A2A (Oncall) Container App Resource Name')

    a2a_weather_a2a_agent_ep = utils.get_deployment_output(output, 'a2AWeatherServerContainerAppFQDN', 'A2A (Weather) Endpoint')
    a2a_oncall_a2a_agent_ep = utils.get_deployment_output(output, 'a2AOncallServerContainerAppFQDN', 'A2A (Oncall) Endpoint')


⚙️ Running: az deployment group show --name mcp-a2a-agents -g lab-mcp-a2a-agents 
✅ Retrieved deployment: mcp-a2a-agents ⌚ 17:47:31.782836 :2s]
👉🏽 APIM Service Id: /subscriptions/77f5679a-f7cd-4b09-91f0-e8808bf4447e/resourceGroups/lab-mcp-a2a-agents/providers/Microsoft.ApiManagement/service/apim-ulp3pavelxqec
👉🏽 APIM Gateway URL: https://apim-ulp3pavelxqec.azure-api.net
👉🏽 APIM Resource Name: apim-ulp3pavelxqec
👉🏽 APIM Subscription Key (masked): ****9b72
👉🏽 Application Insights Name: insights-ulp3pavelxqec
👉🏽 Container Registry Name: acrulp3pavelxqec
👉🏽 Weather Container App Resource Name: aca-weather-ulp3pavelxqec
👉🏽 Oncall Container App Resource Name: aca-oncall-ulp3pavelxqec
👉🏽 GitHub Container App Resource Name: aca-github-ulp3pavelxqec
👉🏽 A2A (Weather) Container App Resource Name: aca-a2a-weather-ulp3pavelxqec
👉🏽 A2A (Oncall) Container App Resource Name: aca-a2a-oncall-ulp3pavelxqec
👉🏽 A2A (Weather) Endpoint: aca-a2a-weather-ulp3pavelxqec.salmontree-acc99004.westeurope.azurecontai

<a id='4'></a>
### 4️⃣ Build and deploy the MCP Servers



In [23]:
build = build + 2 # increment the build number

utils.run(f"az acr build --image {weather_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {weather_mcp_server_src}/Dockerfile {weather_mcp_server_src}/. --no-logs", 
          "Weather MCP Server image was successfully built", "Failed to build the Weather MCP Server image")
utils.run(f'az containerapp update -n {weather_containerapp_resource_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{weather_mcp_server_image}:v0.{build}"', 
          "Weather MCP Server deployment succeeded", "Weather MCP Server deployment failed")

utils.run(f"az acr build --image {oncall_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {oncall_mcp_server_src}/Dockerfile {oncall_mcp_server_src}/. --no-logs", 
          "Oncall MCP Server image was successfully built", "Failed to build the Oncall MCP Server image")
utils.run(f'az containerapp update -n {oncall_containerapp_resource_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{oncall_mcp_server_image}:v0.{build}"', 
          "Oncall MCP Server deployment succeeded", "Oncall MCP Server deployment failed")

utils.run(f"az acr build --image {github_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {github_mcp_server_src}/Dockerfile {github_mcp_server_src}/. --no-logs", 
          "GitHub MCP Server image was successfully built", "Failed to build the GitHub MCP Server image")
utils.run(f'az containerapp update -n {github_containerapp_resource_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{github_mcp_server_image}:v0.{build}"', 
          "GitHub MCP Server deployment succeeded", "GitHub MCP Server deployment failed")

if servicenow_instance_name:
    utils.run(f"az acr build --image {servicenow_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {servicenow_mcp_server_src}/Dockerfile {servicenow_mcp_server_src}/. --no-logs", 
            "ServiceNow MCP Server image was successfully built", "Failed to build the ServiceNow MCP Server image")
    utils.run(f'az containerapp update -n {servicenow_containerapp_resource_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{servicenow_mcp_server_image}:v0.{build}"', 
            "ServiceNow MCP Server deployment succeeded", "ServiceNow MCP Server deployment failed")


⚙️ Running: az acr build --image weather-mcp-server:v0.2 --resource-group lab-mcp-a2a-agents --registry acrulp3pavelxqec --file src/weather/mcp-server/Dockerfile src/weather/mcp-server/. --no-logs 
✅ Weather MCP Server image was successfully built ⌚ 17:48:50.231138 :7s]
⚙️ Running: az containerapp update -n aca-weather-ulp3pavelxqec -g lab-mcp-a2a-agents --image "acrulp3pavelxqec.azurecr.io/weather-mcp-server:v0.2" 
✅ Weather MCP Server deployment succeeded ⌚ 17:49:07.542706 :17s]
⚙️ Running: az acr build --image oncall-mcp-server:v0.2 --resource-group lab-mcp-a2a-agents --registry acrulp3pavelxqec --file src/oncall/mcp-server/Dockerfile src/oncall/mcp-server/. --no-logs 
✅ Oncall MCP Server image was successfully built ⌚ 17:49:42.237003 :34s]
⚙️ Running: az containerapp update -n aca-oncall-ulp3pavelxqec -g lab-mcp-a2a-agents --image "acrulp3pavelxqec.azurecr.io/oncall-mcp-server:v0.2" 
✅ Oncall MCP Server deployment succeeded ⌚ 17:49:59.004087 :16s]
⚙️ Running: az acr build --image g

<a id='testconnection'></a>
### 🧪 Test the connection to the MCP servers and List Tools

💡 To integrate MCP servers in VS Code, use the MCP server URL  `../sse ` for configuration in GitHub Copilot Agent Mode

If the notebook is run again, the JWT validation that gets applied to the policies later on must first be removed. Otherwise, the next calls below will fail as auth is not yet expected to be in place.

In [24]:
policy_xml_file = "src/github/apim-api/no-auth-client-policy.xml"

with open(policy_xml_file, 'r') as file:
    policy_xml = file.read()
    utils.update_api_operation_policy(subscription_id, resource_group_name, apim_resource_name, "github-mcp", "sse", policy_xml)
    utils.update_api_operation_policy(subscription_id, resource_group_name, apim_resource_name, "github-mcp", "messages", policy_xml)

⚙️ Running: az account get-access-token --resource https://management.azure.com/ 
✅ Successfully obtained access token ⌚ 17:57:17.947242 :1s]
Updating the API policy...
Response status: 200 - OK
⚙️ Running: az account get-access-token --resource https://management.azure.com/ 
✅ Successfully obtained access token ⌚ 17:57:19.666519 :0s]
Updating the API policy...
Response status: 200 - OK


In [25]:
import os, json, asyncio, time, requests
from mcp import ClientSession
from mcp.client.sse import sse_client
import nest_asyncio
nest_asyncio.apply()

async def list_tools(server_url, authorization_header = None):
    headers = {"Authorization": authorization_header} if authorization_header else None
    streams = None
    session = None
    tools = []
    try:
        streams_ctx = sse_client(server_url, headers)
        streams = await streams_ctx.__aenter__()
        session_ctx = ClientSession(streams[0], streams[1])
        session = await session_ctx.__aenter__()
        await session.initialize()
        response = await session.list_tools()
        tools = response.tools
    except Exception as e:
        print(f"❌ Error: {e}")
    finally:
        # Ensure session and streams are closed if they were opened
        if session is not None:
            await session_ctx.__aexit__(None, None, None) # type: ignore
        if streams is not None:
            await streams_ctx.__aexit__(None, None, None) # type: ignore
    if tools:
        print(f"✅ Connected to server {server_url}")
        print("⚙️ Tools:")
        for tool in tools:
            print(f"  - {tool.name}")
            print(f"     Input Schema: {tool.inputSchema}")

try:    
    asyncio.run(list_tools(f"{apim_resource_gateway_url}/weather/sse"))
    asyncio.run(list_tools(f"{apim_resource_gateway_url}/github/sse"))
    asyncio.run(list_tools(f"{apim_resource_gateway_url}/oncall/sse"))
    if servicenow_instance_name:
        asyncio.run(list_tools(f"{apim_resource_gateway_url}/servicenow/sse"))
finally:
    print(f"✅ Connection closed")


✅ Connected to server https://apim-ulp3pavelxqec.azure-api.net/weather/sse
⚙️ Tools:
  - get_cities
     Input Schema: {'properties': {'country': {'title': 'Country', 'type': 'string'}}, 'required': ['country'], 'title': 'get_citiesArguments', 'type': 'object'}
  - get_weather
     Input Schema: {'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weatherArguments', 'type': 'object'}
✅ Connected to server https://apim-ulp3pavelxqec.azure-api.net/github/sse
⚙️ Tools:
  - authorize_github
     Input Schema: {'properties': {}, 'title': 'authorize_githubArguments', 'type': 'object'}
  - get_user
     Input Schema: {'properties': {}, 'title': 'get_userArguments', 'type': 'object'}
  - get_issues
     Input Schema: {'properties': {'username': {'title': 'Username', 'type': 'string'}, 'repo': {'title': 'Repo', 'type': 'string'}}, 'required': ['username', 'repo'], 'title': 'get_issuesArguments', 'type': 'object'}
✅ Connected to server https://apim-ul

<a id='inspector'></a>
### 🧪 (optional) Use the [MCP Inspector](https://modelcontextprotocol.io/docs/tools/inspector) for testing and debugging the MCP servers

#### Execute the following steps:
1. Execute `npx @modelcontextprotocol/inspector` in a terminal
2. Open the provided URL in a browser
3. Set the transport type as SSE
4. Provide the MCP server url and click connect
5. Select the "Tools" tab to see and run the available tools

<a id='functioncalling'></a>
### 🧪 Run an OpenAI completion with MCP tools

👉 Both the calls to Azure OpenAI and the MCP tools will be managed through Azure API Management.  


In [26]:
# type: ignore
import json, asyncio
from mcp import ClientSession
from mcp.client.sse import sse_client
from openai import AzureOpenAI
import nest_asyncio
nest_asyncio.apply()

async def call_tool(mcp_session, function_name, function_args):
    try:
        func_response = await mcp_session.call_tool(function_name, function_args)
        func_response_content = func_response.content
    except Exception as e:
        func_response_content = json.dumps({"error": str(e)})
    return str(func_response_content)

async def run_completion_with_tools(server_url, prompt):
    streams = None
    session = None
    try:
        streams_ctx = sse_client(server_url)
        streams = await streams_ctx.__aenter__()
        session_ctx = ClientSession(streams[0], streams[1])
        session = await session_ctx.__aenter__()
        await session.initialize()
        response = await session.list_tools()
        tools = response.tools
        print(f"✅ Connected to server {server_url}")
        openai_tools = [{
                "type": "function",
                "function": {
                    "name": tool.name,
                    "parameters": tool.inputSchema
                },
            } for tool in tools]

        # Step 1: send the conversation and available functions to the model
        print("▶️ Step 1: start a completion to identify the appropriate functions to invoke based on the prompt")
        client = AzureOpenAI(
            azure_endpoint=apim_resource_gateway_url,
            api_key=apim_subscription_key,
            api_version=openai_api_version
        )            
        messages = [ {"role": "user", "content": prompt} ]
        response = client.chat.completions.create(
            model=openai_deployment_name,
            messages=messages,
            tools=openai_tools,
        )
        response_message = response.choices[0].message
        tool_calls = response_message.tool_calls
        if tool_calls:
            # Step 2: call the function
            messages.append(response_message)  # extend conversation with assistant's reply
            # send the info for each function call and function response to the model
            print("▶️ Step 2: call the functions")
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                print(f"   Function Name: {function_name} Function Args: {function_args}")

                function_response = await call_tool(session, function_name, function_args)
                # Add the tool response
                print(f"   Function response: {function_response}")
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )  # extend conversation with function response
            print("▶️ Step 3: finish with a completion to anwser the user prompt using the function response")
            second_response = client.chat.completions.create(
                model=openai_deployment_name,
                messages=messages,
            )  # get a new response from the model where it can see the function response
            print("💬", second_response.choices[0].message.content)
    except Exception as e:
        print(f"❌ Error: {e}")
    finally:
        if session is not None:
            await session_ctx.__aexit__(None, None, None)
        if streams is not None:
            await streams_ctx.__aexit__(None, None, None)

asyncio.run(run_completion_with_tools(f"{apim_resource_gateway_url}/weather/sse", "What's the current weather in Lisbon?"))
asyncio.run(run_completion_with_tools(f"{apim_resource_gateway_url}/oncall/sse", "Who's oncall today?"))


✅ Connected to server https://apim-ulp3pavelxqec.azure-api.net/weather/sse
▶️ Step 1: start a completion to identify the appropriate functions to invoke based on the prompt
▶️ Step 2: call the functions
   Function Name: get_weather Function Args: {'city': 'Lisbon'}
   Function response: [TextContent(type='text', text="{'city': 'Lisbon', 'condition': 'Rainy', 'temperature': 34.89, 'humidity': 89.65}", annotations=None)]
▶️ Step 3: finish with a completion to anwser the user prompt using the function response
💬 The current weather in Lisbon is rainy, with a temperature of approximately 34.9°C and humidity at around 89.7%.
✅ Connected to server https://apim-ulp3pavelxqec.azure-api.net/oncall/sse
▶️ Step 1: start a completion to identify the appropriate functions to invoke based on the prompt
▶️ Step 2: call the functions
   Function Name: get_oncall_list Function Args: {}
   Function response: [TextContent(type='text', text="[{'id': 1, 'firstName': 'Julia', 'lastName': 'Smith', 'alias': 

## Create multiple agents via multiple technologies (Semantic Kernel, Azure AI Agents, Autogen)

## Create A2A Agents (SK & Autogen)

<a id='sk'></a>
### 🧪 Execute a standalone [Semantic Kernel Agent using MCP Tools - Weather agent](https://devblogs.microsoft.com/semantic-kernel/integrating-model-context-protocol-tools-with-semantic-kernel-a-step-by-step-guide/) via Azure API Management

In [27]:
import asyncio
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPSsePlugin

user_input = "What's the current weather in Lisbon?"

async def sk_weather_agent():
    # 1. Create the agent
    async with MCPSsePlugin(
        name="Weather",
        url=f"{apim_resource_gateway_url}/weather/sse",
        description="Weather Plugin",
    ) as weather_plugin:
        agent = ChatCompletionAgent(
            service=AzureChatCompletion(
                endpoint=apim_resource_gateway_url,
                api_key=apim_subscription_key,
                api_version=openai_api_version,                
                deployment_name=openai_deployment_name
            ),
            name="WeatherAgent",
            instructions="Answer questions about the Weather.",
            plugins=[weather_plugin],
        )

        thread: ChatHistoryAgentThread | None = None

        print(f"# User: {user_input}")
        # 2. Invoke the agent for a response
        response = await agent.get_response(messages=user_input, thread=thread)
        print(f"# {response.name}: {response} ")
        thread = response.thread # type: ignore

        # 3. Cleanup: Clear the thread
        await thread.delete() if thread else None

asyncio.run(sk_weather_agent())

# User: What's the current weather in Lisbon?
# WeatherAgent: The current weather in Lisbon is cloudy, with a temperature of 4.21°C and a humidity level of 96.71%. 


<a id='autogen'></a>
### 🧪 Execute a standalone [AutoGen Agent using MCP Tools](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.tools.mcp.html) via Azure API Management

In [34]:
import asyncio
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_ext.tools.mcp import SseMcpToolAdapter, SseServerParams, mcp_server_tools
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken

from pydantic import BaseModel
from typing import Any, Callable, Literal, Optional, Union

class AgentResponse(BaseModel):
    status: Literal["input_required", "completed", "error"]
    message: str

async def ag_oncall_agent(url, prompt) -> None:
    # Create server params for the remote MCP service
    server_params = SseServerParams(
        url=url,
        headers={"Content-Type": "application/json"},
        timeout=30,  # Connection timeout in seconds
    )

    # Get all available tools
    oncall_tools = await mcp_server_tools(server_params)

    for tool in oncall_tools:
        print (tool._strict)
        tool._strict = True

    # Create an agent that can use the translation tool
    model_client = AzureOpenAIChatCompletionClient(azure_deployment=openai_deployment_name, model=openai_deployment_name,
                azure_endpoint=apim_resource_gateway_url,
                api_key=apim_subscription_key,
                api_version=openai_api_version,
    )
    agent = AssistantAgent(
        name="oncall",
        model_client=model_client,
        tools=oncall_tools, # type: ignore
        system_message="You are a helpful assistant.",
        reflect_on_tool_use=True
    )
    await Console(
        agent.run_stream(task=prompt)
    )

asyncio.run(ag_oncall_agent(f"{apim_resource_gateway_url}/oncall/sse", "who's on today??"))


False
---------- TextMessage (user) ----------
who's on today??
---------- ToolCallRequestEvent (oncall) ----------
[FunctionCall(id='call_yIz7HWveYqBckcSfDvgnZMm4', arguments='{}', name='get_oncall_list')]
---------- ToolCallExecutionEvent (oncall) ----------
[FunctionExecutionResult(content='[{"type": "text", "text": "[{\'id\': 1, \'firstName\': \'Julia\', \'lastName\': \'Smith\', \'alias\': \'jsmith\', \'status\': \'on\', \'timezone\': \'PST\'}, {\'id\': 2, \'firstName\': \'Alex\', \'lastName\': \'Johnson\', \'alias\': \'ajohnson\', \'status\': \'on\', \'timezone\': \'EST\'}, {\'id\': 3, \'firstName\': \'Maria\', \'lastName\': \'Garcia\', \'alias\': \'mgarcia\', \'status\': \'off\', \'timezone\': \'CET\'}, {\'id\': 4, \'firstName\': \'David\', \'lastName\': \'Wilson\', \'alias\': \'dwilson\', \'status\': \'on\', \'timezone\': \'CET\'}, {\'id\': 5, \'firstName\': \'Sarah\', \'lastName\': \'Chen\', \'alias\': \'schen\', \'status\': \'on\', \'timezone\': \'CET\'}, {\'id\': 6, \'firstNa

### Building Customisable A2A MCP-enabled Agent images for deployment in ACA

#### Semantic-Kernel

In [36]:
build = build + 1

a2a_sk_server_image = "a2a-sk-server"
a2a_sk_server_src = "src/a2a_servers/a2a_sk_mcp_agent"

utils.run(f"az acr build --image {a2a_sk_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {a2a_sk_server_src}/Dockerfile {a2a_sk_server_src}/. --no-logs", 
          "Generic A2A SK Server with MCP image was successfully built", "Failed to build the Generic A2A SK Server with MCP image")

⚙️ Running: az acr build --image a2a-sk-server:v0.4 --resource-group lab-mcp-a2a-agents --registry acrulp3pavelxqec --file src/a2a_servers/a2a_sk_mcp_agent/Dockerfile src/a2a_servers/a2a_sk_mcp_agent/. --no-logs 
✅ Generic A2A SK Server with MCP image was successfully built ⌚ 18:34:54.007679 :35s]


#### Autogen

In [ ]:
a2a_ag_server_image = "a2a-ag-server"
a2a_ag_server_src = "src/a2a_servers/a2a_ag_mcp_agent"

utils.run(f"az acr build --image {a2a_ag_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {a2a_ag_server_src}/Dockerfile {a2a_ag_server_src}/. --no-logs", 
          "Generic A2A Autogen Server with MCP image was successfully built", "Failed to build the Generic A2A Autogen Server with MCP image")

⚙️ Running: az acr build --image a2a-ag-server:v0.4 --resource-group lab-mcp-a2a-agents --registry acrulp3pavelxqec --file src/a2a_servers/a2a_ag_mcp_agent/Dockerfile src/a2a_servers/a2a_ag_mcp_agent/. --no-logs 


### 🧪 Deploying A2A Agents in ACA

Use the following parameters to customise the deployment:
title, mcp_url

In [ ]:
title="Weather"
mcp_url = "/weather/sse"

utils.run(f'az containerapp secret set -n  {a2a_sk_weather_containerapp_resource_name} -g {resource_group_name} --secrets apimsubscriptionkey={apim_subscription_key}', "Weather A2A Server secret updated", "Weather A2A Server secret update failed")
utils.run(f'az containerapp update -n {a2a_sk_weather_containerapp_resource_name} -g  {resource_group_name} --image "{container_registry_name}.azurecr.io/{a2a_sk_server_image}:v0.{build}" --set-env-vars TITLE={title} MCP_URL={mcp_url} A2A_URL={apim_resource_gateway_url}/a2a-weather APIM_GATEWAY_URL={apim_resource_gateway_url} OPENAI_API_VERSION={openai_api_version} OPENAI_DEPLOYMENT_NAME={openai_deployment_name} APIM_SUBSCRIPTION_KEY=secretref:apimsubscriptionkey', 
          "Weather A2A SK Server with MCP deployment succeeded", "Weather A2A SK Server with MCP deployment failed")

In [ ]:
utils.print_info(f'A2A URL for Weather Agent: {apim_resource_gateway_url}/a2a-weather')

In [ ]:
title="Oncall"
mcp_url = "/oncall/sse"

utils.run(f'az containerapp secret set -n  {a2a_oncall_containerapp_resource_name} -g {resource_group_name} --secrets apimsubscriptionkey={apim_subscription_key}', f"{title} A2A Server secret updated", f"{title} A2A Server secret update failed")
utils.run(f'az containerapp update -n {a2a_oncall_containerapp_resource_name} -g  {resource_group_name} --image "{container_registry_name}.azurecr.io/{a2a_ag_server_image}:v0.{build}" --set-env-vars TITLE={title} MCP_URL={mcp_url} A2A_URL={apim_resource_gateway_url}/a2a-oncall APIM_GATEWAY_URL={apim_resource_gateway_url} OPENAI_API_VERSION={openai_api_version} OPENAI_DEPLOYMENT_NAME={openai_deployment_name} APIM_SUBSCRIPTION_KEY=secretref:apimsubscriptionkey', 
          "Oncall A2A SK Server with MCP deployment succeeded", "Oncall A2A SK Server with MCP deployment failed")

In [ ]:
utils.print_info(f'A2A URL for Oncall Agent: {apim_resource_gateway_url}/a2a-oncall')

<a id='githubconfig'></a>
### 5️⃣ Create a GitHub OAuth app and configure the credential provider

#### Step 1 - [Register the application in GitHub](https://learn.microsoft.com/en-us/azure/api-management/credentials-how-to-github#step-1-register-an-application-in-github)

👉 Use the Authorization callback URL that is provided below  
👉 Copy the Client ID and Client secret

#### Step 2 - [Configure the credential provider in API Management](https://learn.microsoft.com/en-us/azure/api-management/credentials-how-to-github#step-2-configure-a-credential-provider-in-api-management)

👉 You just need to update the Client ID and Client secret on the existing `github` credential manager provider  
👉 Disregard the remaining steps outlined in the documentation


<a id='servicenowconfig'></a>
### 6️⃣ Create a ServiceNow OAuth app and configure the credential provider

ℹ️ If you do not wish to use ServiceNow, please skip these steps

#### Step 1 - [Register the application in ServiceNow](https://www.servicenow.com/docs/bundle/yokohama-application-development/page/build/pipelines-and-deployments/task/create-oauth-api-endpoints-for-external-clients.html)

👉 Use the Authorization callback URL that is provided bellow  
👉 Copy the Client ID and Client secret

#### Step 2 - Configure the credential provider in API Management

👉 You just need to update the Client ID and Client secret on the existing `servicenonw` credential manager provider  

In [ ]:
print(f"Authorization callback URL: https://authorization-manager.consent.azure-apim.net/redirect/apim/{apim_resource_name}")

<a id='githubtest'></a>
### 🧪 Run the GitHub MCP Server with VS Code to retrieve GitHub Issues

1. [Configure the GitHub MCP Server in VS Code](https://code.visualstudio.com/docs/copilot/chat/mcp-servers#_add-an-mcp-server) 
2. Type in the chat the following prompt: `Please list all the issues assigned to me in the GitHub repository {your-repo-name} under the organization {your-org-name}`
3. The agent will suggest running the `authorize_github` tool.
4. Once the user accepts to run the tool, the agent will call the `authorize_github` and provide an URL to proceed with the authentication and authorization on GitHub.
5. After the user confirms that it's done, the agent will suggest running the `get_user` tool.
6. Once the user accepts to run the `get_user` tool, the agent will call the tool, return user information as context and suggest running the `get_issues` tool.
7. Once the user accepts to run the `get_issues` tool, the agent will provide the list of issues from the given repo.

<a id='servicenowtest'></a>
### 🧪 Run the ServiceNow MCP Server with VS Code to manage ServiceNow incidents

1. [Configure the ServiceNow MCP Server in VS Code](https://code.visualstudio.com/docs/copilot/chat/mcp-servers#_add-an-mcp-server) 
2. Type in the chat the following prompt: `Please list my servicenow incidents`
3. The agent will suggest running the `authorize_servicenow` tool.
4. Once the user accepts to run the tool, the agent will call the `authorize_servicenow` and provide an URL to proceed with the authentication and authorization on ServiceNow.
5. After the user confirms that it's done, the agent will suggest running the `list_incidents` tool.
6. Once the user accepts to run the `list_incidents` tool, the agent will provide the list of incidents for the connected ServiceNow instance.
7. You can also retrieve details for a specific incident or create a new one.

✨ Type in the chat the following prompt: `Create a ServiceNow incident for each GitHub issue`. To combine GitHub and ServiceNow MCP Servers.

<a id='validate-jwt'></a>
### 🔐 (Optional) Implement [authorization policies](src/github/apim-api/auth-client-policy.xml) on MCP endpoints

👉 To ensure the enforcement of valid security tokens, we apply the `validate-jwt` policy to the `/sse` and `/messages` endpoints. The following code snippet demonstrates the application of this policy to GitHub API operations for token validation:

In [ ]:
policy_xml_file = "src/github/apim-api/auth-client-policy.xml"

with open(policy_xml_file, 'r') as file:
    policy_xml = file.read()
    utils.update_api_operation_policy(subscription_id, resource_group_name, apim_resource_name, "github-mcp", "sse", policy_xml)
    utils.update_api_operation_policy(subscription_id, resource_group_name, apim_resource_name, "github-mcp", "messages", policy_xml)

<a id='unauthorizedtest'></a>
### 🧪 Test the authorization **WITHOUT** a valid token

In [ ]:
# Unauthenticated call should fail with 401 Unauthorized
import requests
utils.print_info("Calling sse endpoint WITHOUT authorization...")
response = requests.get(f"{apim_resource_gateway_url}/github/sse", headers={"Content-Type": "application/json"})
if response.status_code == 401:
    utils.print_ok("Received 401 Unauthorized as expected")
elif response.status_code == 200:
    utils.print_error("Call succeeded. Double check that validate-jwt policy has been deployed to sse endpoint")
else:
    utils.print_error(f"Unexpected status code: {response.status_code}")


<a id='authorizedtest'></a>
### 🧪 Test the authorization **WITH** a valid token

In [ ]:
import requests
# Authenticated call should succeed
utils.print_info("Calling sse endpoint WITH authorization...")
output = utils.run("az account get-access-token --resource \"https://azure-api.net/authorization-manager\"")
if output.success and output.json_data:
    access_token = output.json_data.get('accessToken')
    response = requests.get(f"{apim_resource_gateway_url}/github/sse", stream=True,
                            headers={"Content-Type": "application/json", "Authorization": "Bearer " + str(access_token)})
    if response.status_code == 200:
        utils.print_ok("Received status code 200 as expected")
    else:
        utils.print_error(f"Unexpected status code: {response.status_code}")



<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.